In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
test=pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

In [ ]:
train.head()

In [ ]:
train=train.drop(train.columns[0:2],1)
test=test.drop(test.columns[0:2],1)

In [ ]:
train.shape, test.shape

In [ ]:
train.describe()

In [ ]:
def get_cat(df):
    """Gets list of categorical features from passed dataframe"""
    cat=[]
    for i in df.columns:
        if df[i].dtypes=='object':
            cat.append(i) 
    return cat

In [ ]:
cat1=get_cat(train)
cat2=get_cat(test)

In [ ]:
train[train.isna().any(1)].shape

In [ ]:
test[test.isna().any(1)].shape

## handling missing nominal features

In [ ]:
def get_nom(df):
    """Gets nominal feature"""
    nom=[]
    for i in df.columns:
        if df[i].dtypes!='object':
            nom.append(i)
    return nom[2:]  # no need of feature id and age

In [ ]:
nom1=get_nom(train)
nom2=get_nom(test)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp=SimpleImputer(strategy='median')
# impute with median

In [ ]:
def impute(df, nom):
    "Impute for nan values"
    for i in nom:
        df[i]=imp.fit_transform(df[i][:, np.newaxis])
    return df

In [ ]:
train=impute(train, nom1)
test=impute(test, nom2)

In [ ]:
train[train.isna().any(1)].shape, test[test.isna().any(1)].shape
# no missing values

### Visulaization, lets look what our data intend to tell us

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def bar_plot(df,col):
    """Plot bar graph based on passed categorical features"""
    
    satisfied=df[col][df['satisfaction']=='satisfied'].value_counts()
    neutral=df[col][df['satisfaction']=='neutral or dissatisfied'].value_counts()
    dt=pd.DataFrame([satisfied,neutral], index=['satisfied','neutral or dissatisfied'])
    dt.plot.bar(stacked=True)

In [ ]:
bar_plot(train,'Class')

In [ ]:
bar_plot(train, 'Gender')

In [ ]:
bar_plot(train, 'Type of Travel')

## feature encoding for categorical features

In [ ]:
cat1=cat1[0:4]
cat2=cat2[0:4]

In [ ]:
def cat_enc(df,cat):
    """Encoding (by get_dummies) for categorical features.
    Dropping the original feature before getting new dummy for each category in feature
    """
    dummies=[]
    for i in cat:
        dummies=pd.get_dummies(df[i], drop_first=True, prefix=i)
        df=df.drop(i,axis=1)
        df=pd.concat([df,dummies],1)
    return df

In [ ]:
train=cat_enc(train, cat1)
test=cat_enc(test, cat2)

In [ ]:
train.head(1)

In [ ]:
# encoding for our target feature (satisfaction)
train['satisfaction']=train['satisfaction'].map({'satisfied':1,'neutral or dissatisfied':0})
test['satisfaction']=test['satisfaction'].map({'satisfied':1,'neutral or dissatisfied':0})
# no need to target feature in test dataframe since we are about to predict it
#test.drop('satisfaction',1,inplace=True)

In [ ]:
dtrain=train.copy()
dtest=test.copy()

## Model validation

In [ ]:
train.head(1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X=dtrain.drop('satisfaction',1)
y=dtrain['satisfaction']
x_test=dtest.drop('satisfaction',1)

### Scaling

In [ ]:
scale=StandardScaler()
X=scale.fit_transform(X)
x_test=scale.fit_transform(x_test)

In [ ]:
x_train, x_val, y_train, y_val=train_test_split(X,y,shuffle=False)

#### Cross Validation using stratifiedkfold 

In [ ]:
skf=StratifiedKFold(n_splits=10, shuffle=False)

### LogisticRegression

In [ ]:
logi_reg=LogisticRegression()
score= cross_val_score(logi_reg, X, y, scoring='accuracy', cv=skf)
print(score)
print('avg score:',np.mean(score)*100)

### RandomForestClassifier

In [ ]:
rfc=RandomForestClassifier(5)
score= cross_val_score(rfc, X, y, scoring='accuracy', cv=skf)
print(score)
print('avg score:',np.mean(score)*100)

### GradientBoostingClassifier

In [ ]:
gbc=GradientBoostingClassifier(n_estimators=20)
score= cross_val_score(gbc, X, y, scoring='accuracy', cv=skf)
print(score)
print('avg score:',np.mean(score)*100)

### Testing

In [ ]:
## rfc looks good here to me
## lets predict using this
rfc=RandomForestClassifier(5)
rfc.fit(x_train, y_train)
ypred=rfc.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(dtest['satisfaction'], ypred)